<a href="https://colab.research.google.com/github/myidispg/NLP-Projects/blob/master/Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My first Google Colab notebook for Neural Machine Translation in Pytorch.
---



### **The necessary imports and running the code on GPU if available.**
GPU is necessary for faster model training.

In [1]:

import requests
import tarfile
from os import path

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SOS_token = 0
EOS_token = 1

device

device(type='cuda')

##Download and save the dataset

In [2]:
dataset_url = 'http://www.cfilt.iitb.ac.in/iitb_parallel/iitb_corpus_download/parallel.tgz'
tgz_file = requests.get(dataset_url, stream=True)

with open("parallel.tgz", "wb") as f:
  for chunk in tgz_file.iter_content(chunk_size=1024):
    if chunk:
      f.write(chunk)
      
if path.exists("parallel.tgz"):
  print('File saved successfully.')


File saved successfully.


In [3]:
data_list = []

tar = tarfile.open("parallel.tgz")
for member in tar.getmembers():
  f = tar.extractfile(member)
  if f is not None:
    data_list.append(f)

data_list
  

[<ExFileObject name='parallel.tgz'>, <ExFileObject name='parallel.tgz'>]

### Define a class for a Language
This class will contain the word2idx, idx2word, number of words in the vocabulary and max sentence length of that language.

In [0]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2
        self.max_sent_length = 1
        
    def addSentence(self, sentence):
        sent_length = len(sentence.split(' '))
        self.max_sent_length = sent_length if sent_length > self.max_sent_length else self.max_sent_length        
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
hindi_lang = Lang('hindi')
english_lang = Lang('english')


###Some helper functions to read the data, create pairs and the language vocabularies.

In [5]:
def addWordsToLang(lang, lines):
    for line in lines:
        lang.addSentence(line)
    
    return lang

def create_pairs(lang1, lang2):
    pairs = []

    for lang1_sent, lang2_sent in zip(lang1, lang2):
        pairs.append([lang1_sent, lang2_sent])
        
    return pairs

def createLanguagesAndPairs(data_list, lang1, lang2):
    print('Opening files and reading the sentences')
      
    lang1_lines = data_list[0].read().strip().split('\n')
    lang2_lines = data_list[1].read().strip().split('\n')
    
    print('Creating pairs...')
    pairs = create_pairs(lang1_lines, lang2_lines)
    
    print('Adding words to languages')
    lang1 = addWordsToLang(lang1, lang1_lines)
    lang2 = addWordsToLang(lang2, lang2_lines)
    
    print('Done creating languages')
    
    return pairs, lang1, lang2
  
pairs, hindi_lang, english_lang = createLanguagesAndPairs(data_list, hindi_lang, english_lang)

Opening files and reading the sentences


TypeError: ignored